In [38]:
import pandas as pd
import json
import sqlalchemy

In [46]:
from sqlalchemy import create_engine, inspect, text, MetaData
from sqlalchemy.orm import declarative_base

In [ ]:
dados_fornecedores = pd.read_csv('fornecedores.csv', sep=';', dtype={'cod_fornecedor': str}, thousands='.', decimal=',', encoding='latin-1')

In [14]:
dados_fornecedores

,cod_fornecedor,razao_social,cidade,uf,receita_anual
0,F-201,Distribuidora Elï¿½trica Sï¿½o Paulo Ltda,Sï¿½o Paulo,SP,"2.500.000,00"
1,F-305,Eletropeï¿½as Mineira ME,Belo Horizonte,MG,"850.000,00"
2,F-118,Casa do Eletricista Conceiï¿½ï¿½o & Filhos,Curitiba,PR,"1.200.000,00"
3,F-410,Iluminaï¿½ï¿½o Nordeste S.A.,Recife,PE,"3.100.000,00"
4,F-500,Materiais Rï¿½pidos Ltda,Manaus,AM,"420.000,00"


In [15]:
dados_fornecedores.dtypes

cod_fornecedor    str
razao_social      str
cidade            str
uf                str
receita_anual     str
dtype: object

In [16]:
dados_fornecedores['receita_anual'] = dados_fornecedores['receita_anual'].str.replace('.', '', regex=False).str.replace(',', '.').astype('float64')

In [18]:
dados_fornecedores['receita_anual']

0    2500000.0
1     850000.0
2    1200000.0
3    3100000.0
4     420000.0
Name: receita_anual, dtype: float64

In [19]:
dados_fornecedores.dtypes

cod_fornecedor        str
razao_social          str
cidade                str
uf                    str
receita_anual     float64
dtype: object

In [20]:
dados_fornecedores

,cod_fornecedor,razao_social,cidade,uf,receita_anual
0,F-201,Distribuidora Elï¿½trica Sï¿½o Paulo Ltda,Sï¿½o Paulo,SP,2500000.0
1,F-305,Eletropeï¿½as Mineira ME,Belo Horizonte,MG,850000.0
2,F-118,Casa do Eletricista Conceiï¿½ï¿½o & Filhos,Curitiba,PR,1200000.0
3,F-410,Iluminaï¿½ï¿½o Nordeste S.A.,Recife,PE,3100000.0
4,F-500,Materiais Rï¿½pidos Ltda,Manaus,AM,420000.0


In [22]:
with open('contratos.json', '+r') as file:
    dados_contratos = json.loads(file.read())

In [24]:
dados_contratos = pd.read_json('contratos.json', orient='records', dtype={'cod_fornecedor': str})

In [25]:
dados_contratos

,cod_fornecedor,num_contrato,vigencia_inicio,vigencia_fim,valor_contrato,status_contrato
0,F-201,CTR-2024-001,01/01/2024,31/12/2025,250000,ativo
1,F-305,CTR-2024-015,01/03/2024,28/02/2026,180000,ativo
2,F-118,CTR-2024-022,15/06/2024,14/06/2026,320000,ativo
3,F-410,CTR-2024-033,01/09/2024,31/08/2026,95000,suspenso
4,F-201,CTR-2025-002,01/01/2025,31/12/2025,175000,ativo


In [26]:
dados_contratos.dtypes

cod_fornecedor       str
num_contrato         str
vigencia_inicio      str
vigencia_fim         str
valor_contrato     int64
status_contrato      str
dtype: object

In [29]:
dados_contratos['vigencia_inicio'] = pd.to_datetime(dados_contratos['vigencia_inicio'], format='%d/%m/%Y')
dados_contratos['vigencia_fim'] = pd.to_datetime(dados_contratos['vigencia_fim'], format='%d/%m/%Y')

In [30]:
dados_contratos.dtypes

cod_fornecedor                str
num_contrato                  str
vigencia_inicio    datetime64[us]
vigencia_fim       datetime64[us]
valor_contrato              int64
status_contrato               str
dtype: object

In [31]:
dados_contratos

,cod_fornecedor,num_contrato,vigencia_inicio,vigencia_fim,valor_contrato,status_contrato
0,F-201,CTR-2024-001,2024-01-01,2025-12-31,250000,ativo
1,F-305,CTR-2024-015,2024-03-01,2026-02-28,180000,ativo
2,F-118,CTR-2024-022,2024-06-15,2026-06-14,320000,ativo
3,F-410,CTR-2024-033,2024-09-01,2026-08-31,95000,suspenso
4,F-201,CTR-2025-002,2025-01-01,2025-12-31,175000,ativo


In [32]:
dados_notas_fiscais = pd.read_xml('notas_fiscais.xml', dtype={'cod_fornecedor': str}, encoding='utf-8')

In [33]:
dados_notas_fiscais

,numero_nf,cod_fornecedor,data_emissao,valor_nf,num_contrato
0,NF-10201,F-201,2025-01-15,45000.0,CTR-2024-001
1,NF-10202,F-201,2025-02-10,32000.0,CTR-2025-002
2,NF-10305,F-305,2025-01-20,28000.0,CTR-2024-015
3,NF-10118,F-118,2025-01-25,55000.0,CTR-2024-022
4,NF-10119,F-118,2025-02-08,61000.0,CTR-2024-022
5,NF-10120,F-118,2025-02-12,48000.0,CTR-2024-022


In [34]:
dados_notas_fiscais.dtypes

numero_nf             str
cod_fornecedor        str
data_emissao          str
valor_nf          float64
num_contrato          str
dtype: object

In [35]:
dados_notas_fiscais['data_emissao'] = pd.to_datetime(dados_notas_fiscais['data_emissao'], format='%Y-%m-%d')

In [36]:
dados_notas_fiscais.dtypes

numero_nf                    str
cod_fornecedor               str
data_emissao      datetime64[us]
valor_nf                 float64
num_contrato                 str
dtype: object

In [37]:
dados_notas_fiscais

,numero_nf,cod_fornecedor,data_emissao,valor_nf,num_contrato
0,NF-10201,F-201,2025-01-15,45000.0,CTR-2024-001
1,NF-10202,F-201,2025-02-10,32000.0,CTR-2025-002
2,NF-10305,F-305,2025-01-20,28000.0,CTR-2024-015
3,NF-10118,F-118,2025-01-25,55000.0,CTR-2024-022
4,NF-10119,F-118,2025-02-08,61000.0,CTR-2024-022
5,NF-10120,F-118,2025-02-12,48000.0,CTR-2024-022


In [47]:
engine = create_engine('sqlite:///auditoria.db')
Base = declarative_base()


In [48]:
Base.metadata.create_all(engine)

In [52]:
dados_fornecedores.to_sql('fornecedores', con=engine, if_exists='replace', index=False)

5

In [53]:
pd.read_sql_table('fornecedores', engine)

,cod_fornecedor,razao_social,cidade,uf,receita_anual
0,F-201,Distribuidora Elï¿½trica Sï¿½o Paulo Ltda,Sï¿½o Paulo,SP,2500000.0
1,F-305,Eletropeï¿½as Mineira ME,Belo Horizonte,MG,850000.0
2,F-118,Casa do Eletricista Conceiï¿½ï¿½o & Filhos,Curitiba,PR,1200000.0
3,F-410,Iluminaï¿½ï¿½o Nordeste S.A.,Recife,PE,3100000.0
4,F-500,Materiais Rï¿½pidos Ltda,Manaus,AM,420000.0


In [54]:
dados_contratos.to_sql('contratos', con=engine, if_exists='replace', index=False)

5

In [55]:
pd.read_sql_table('contratos', engine)

,cod_fornecedor,num_contrato,vigencia_inicio,vigencia_fim,valor_contrato,status_contrato
0,F-201,CTR-2024-001,2024-01-01,2025-12-31,250000,ativo
1,F-305,CTR-2024-015,2024-03-01,2026-02-28,180000,ativo
2,F-118,CTR-2024-022,2024-06-15,2026-06-14,320000,ativo
3,F-410,CTR-2024-033,2024-09-01,2026-08-31,95000,suspenso
4,F-201,CTR-2025-002,2025-01-01,2025-12-31,175000,ativo


In [58]:
dados_notas_fiscais.to_sql('notas_fiscais', con=engine, if_exists='replace', index=False)

6

In [59]:
pd.read_sql_table('notas_fiscais', engine)

,numero_nf,cod_fornecedor,data_emissao,valor_nf,num_contrato
0,NF-10201,F-201,2025-01-15,45000.0,CTR-2024-001
1,NF-10202,F-201,2025-02-10,32000.0,CTR-2025-002
2,NF-10305,F-305,2025-01-20,28000.0,CTR-2024-015
3,NF-10118,F-118,2025-01-25,55000.0,CTR-2024-022
4,NF-10119,F-118,2025-02-08,61000.0,CTR-2024-022
5,NF-10120,F-118,2025-02-12,48000.0,CTR-2024-022


In [60]:
inspector = inspect(engine)

In [61]:
print(inspector.get_table_names())

['contratos', 'fornecedores', 'notas_fiscais']


In [66]:
query = "SELECT f.cod_fornecedor, f.razao_social, SUM(nf.valor_nf) as total_faturado \
        FROM fornecedores f \
        LEFT JOIN notas_fiscais nf ON f.cod_fornecedor = nf.cod_fornecedor \
        GROUP BY f.cod_fornecedor, f.razao_social \
        ORDER BY total_faturado DESC"

In [68]:
print(pd.read_sql(query, engine))

  cod_fornecedor                                razao_social  total_faturado
0          F-118  Casa do Eletricista Conceiï¿½ï¿½o & Filhos        164000.0
1          F-201   Distribuidora Elï¿½trica Sï¿½o Paulo Ltda         77000.0
2          F-305                    Eletropeï¿½as Mineira ME         28000.0
3          F-410                Iluminaï¿½ï¿½o Nordeste S.A.             NaN
4          F-500                    Materiais Rï¿½pidos Ltda             NaN


In [69]:
query = "SELECT c.num_contrato, f.razao_social, c.valor_contrato, \
            COALESCE(SUM(nf.valor_nf), 0) as total_executado, \
            ROUND(COALESCE(SUM(nf.valor_nf), 0) * 100.0 / c.valor_contrato, 1) as pct_executado \
        FROM contratos c \
        JOIN fornecedores f ON c.cod_fornecedor = f.cod_fornecedor \
        LEFT JOIN notas_fiscais nf ON c.num_contrato = nf.num_contrato \
        WHERE c.status_contrato = 'ativo' \
        GROUP BY c.num_contrato \
        ORDER BY pct_executado DESC"

In [70]:
print(pd.read_sql(query, engine))

   num_contrato                                razao_social  valor_contrato  \
0  CTR-2024-022  Casa do Eletricista Conceiï¿½ï¿½o & Filhos          320000   
1  CTR-2025-002   Distribuidora Elï¿½trica Sï¿½o Paulo Ltda          175000   
2  CTR-2024-001   Distribuidora Elï¿½trica Sï¿½o Paulo Ltda          250000   
3  CTR-2024-015                    Eletropeï¿½as Mineira ME          180000   

   total_executado  pct_executado  
0         164000.0           51.3  
1          32000.0           18.3  
2          45000.0           18.0  
3          28000.0           15.6  


In [71]:
query = "SELECT f.cod_fornecedor, f.razao_social, f.uf \
        FROM fornecedores f \
        LEFT JOIN notas_fiscais nf ON f.cod_fornecedor = nf.cod_fornecedor \
        WHERE nf.numero_nf IS NULL"

In [72]:
print(pd.read_sql(query, engine))

  cod_fornecedor                  razao_social  uf
0          F-410  Iluminaï¿½ï¿½o Nordeste S.A.  PE
1          F-500      Materiais Rï¿½pidos Ltda  AM
